<a href="https://colab.research.google.com/github/hejnal/kschool-marketing-digital-geo-bqml/blob/main/colab/exercise2_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis EDA para el conjunto de datos de Spotify

## Instrucciones

Usa Jupyter notebook y bibliotecas estándar para analizar los datos y generar gráficos.

Para descargar el consumo de memoria, se pueden usar BigQuery DataFrames en lugar de Pandas normal.

## Instalar e importar bibliotecas

In [ ]:
!pip install --user --upgrade --quiet bigframes plotly yellowbrick scikit-learn

In [ ]:
import seaborn as sns
from yellowbrick.target import FeatureCorrelation
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

sns.set(rc={'figure.figsize':(11.7,8.27)})

## [Solo en Colab] Autenticar

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Configurar proyecto, región y nombre de tabla

In [ ]:
import bigframes.pandas as bpd

PROJECT_ID = "clean-silo-405314"  # @param {type:"string"}
REGION = "US"  # @param {type:"string"}
bpd.close_session()

# Configurar opciones de BigQuery DataFrames
# Nota: La opción de proyecto no es necesaria en todos los entornos.
# En BigQuery Studio, el ID del proyecto se detecta automáticamente.
bpd.options.bigquery.project = PROJECT_ID

# Nota: La opción de ubicación no es necesaria.
# Por defecto, utiliza la ubicación de la primera tabla o consulta
# pasada a read_gbq(). Para APIs donde no se puede detectar una ubicación,
# automáticamente, la ubicación por defecto es "US".

## Cargar datos directamente desde BigQuery, usando funciones mágicas de bigquery o BigQuery DataFrames

### Opción BigFrames - todas las agregaciones se realizan en BigQuery

In [ ]:
df = bpd.read_gbq('raw_data.spotify_full_dataset', columns=["artist_name", "track_name", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", "loudness", "mode", "popularity", "speechiness", "tempo", "valence", "year"], use_cache=False)


### Opción de palabra clave mágica - descargar datos a Pandas, procesar datos en la memoria local

In [ ]:
# Cargar la extensión mágica de BigQuery
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery df --project $PROJECT_ID --no_query_cache
SELECT
  artist_name,
  track_name,
  popularity,
  year,
  genre,
  danceability,
  energy,
  key,
  loudness,
  mode,
  speechiness,
  acousticness,
  instrumentalness,
  liveness,
  valence,
  tempo,
  duration_ms,
  time_signature
FROM
  `raw_data.spotify_full_dataset`

## Explorar el conjunto de datos de Spotify

### Describir el dataframe

In [ ]:
df.describe()

### Inspeccionar los datos

In [ ]:
# mostrar las primeras 5 filas
df.head()

Comprobemos los valores nulos

In [ ]:
df.isnull().sum()

Veamos las estadísticas de todas las características

In [ ]:
df_stats = df.describe()
df_stats = df_stats.transpose()
df_stats

In [ ]:
df.dtypes

### Histogramas

Veamos los gráficos de popularidad para artistas después de 2010.

In [ ]:
df_filtered = df.loc[df['year'] > 2010]

sns.set(rc={'figure.figsize':(14.7,8.27)})
sns.histplot(df_filtered['popularity'], kde=False)

Popularidad sin valores atípicos.

In [ ]:
from scipy import stats
numeric_features = df.select_dtypes(np.number)
numeric_features_filtered = numeric_features.loc[(numeric_features['year'] >= 2010) & (numeric_features['popularity'] > 0)]
numeric_features_filtered['popularity']
np.abs(stats.zscore(np.array(numeric_features_filtered['popularity'], dtype=np.float64)))

numeric_features_with_no_outliers = numeric_features_filtered[(np.abs(stats.zscore(np.array(numeric_features_filtered['popularity'], dtype=np.float64))) < 3)]

sns.set(rc={'figure.figsize':(14.7,8.27)})
sns.histplot(numeric_features_with_no_outliers['popularity'], kde=False)

Analizar los últimos 3 años.

In [ ]:
df_filtered = df.loc[(df['year'] >= 2020) & (df['year'] <= 2023)]

Analizar el número de canciones por década.

In [ ]:
def get_decade(year):
    period_start = int(year/10) * 10
    decade = '{}s'.format(period_start)
    return decade

df['decade'] = df['year'].apply(get_decade)

sns.displot(df['decade'])

### Correlación entre características

In [ ]:
# Filtrar a columnas numéricas
numeric_columns = df.select_dtypes(include=np.number).columns
df_numeric = df[numeric_columns]

sns.set(rc={'figure.figsize':(12.7,8.27)})
# Calcular correlación y trazar mapa de calor
sns.heatmap(df_numeric.corr())

Correlaciones más avanzadas: energía y popularidad, para diferentes modos - mayor y menor (azul naranja) para cada año por separado.

In [ ]:
sns.set_theme()
sns.set(rc={'figure.figsize':(12.7,8.27)})
sns.relplot(data=df_filtered, x='energy', y='popularity', height=10, aspect=2, hue='mode', col='year', col_wrap=2)

Correlación más básica en el gráfico de barras.

In [ ]:
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
                 'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms', 'key', 'mode']

X, y = df[feature_names], df['popularity']

# Convertir columnas Int64 a float64 como opción segura
for col in ['duration_ms', 'key', 'mode']:
    if col in X.columns and X[col].dtype == 'Int64':
        X[col] = X[col].astype(np.float64) # Usar float64 para ser consistente

# Crear una lista de los nombres de las características (ya hecho, pero se mantiene para contexto)
features = np.array(feature_names)

# Instanciar el visualizador
visualizer = FeatureCorrelation(labels=features)

plt.rcParams['figure.figsize']=(15,15)
visualizer.fit(X, y)     # Ajustar los datos al visualizador
visualizer.show()


### Series temporales

In [ ]:
numeric_features = df.select_dtypes(np.number)

In [ ]:
features_by_year = numeric_features.groupby("year", as_index=False).mean()
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']

# Visualizar la evolución de las características a lo largo del tiempo
fig = px.line(features_by_year, x='year', y=sound_features, height=1000, width=1800)
fig.show()

## Ejercicios
Para tu artista favorito, obtén algunas estadísticas interesantes sobre su carrera, cómo evolucionan sus canciones a lo largo del tiempo y qué las hace exitosas.

Encuentra respuestas a las siguientes preguntas:

* Ver la evolución de las características a lo largo del tiempo.
* ¿En qué años publicaron sus canciones (álbumes) y cuántas canciones se lanzaron?
* ¿Cuál es la canción más popular del artista?
* ¿En qué año se lanzaron las canciones con los niveles de energía promedio más altos?
* ¿Cuál es el nombre de la canción más bailable de tu artista favorito (el del grupo)?
* ¿Qué característica tiene la mayor correlación con la popularidad de la canción?

In [ ]:
# @title Función de ayuda
# Función de ayuda para comparar resultados de Pandas y SQL
def compare_results(pandas_df, sql_df, id_column=None):
    """
    Compara DataFrames básicos de pandas y SQL con suposiciones mínimas.

    Argumentos:
        pandas_df: DataFrame de una operación de pandas
        sql_df: DataFrame de una consulta SQL
        id_column: Columna para ordenar la comparación (como 'year', 'track_name', etc.)
    """
    # 1. Comprobar si las formas coinciden
    if pandas_df.shape != sql_df.shape:
        print(f"Discrepancia de forma: Pandas {pandas_df.shape}, SQL {sql_df.shape}")
        return False

    # 2. Asegurarse de que las columnas coinciden (ignorando el orden)
    pandas_cols = set(pandas_df.columns)
    sql_cols = set(sql_df.columns)
    if pandas_cols != sql_cols:
        print(f"Discrepancia de columnas: Solo en Pandas {pandas_cols - sql_cols}, Solo en SQL {sql_cols - pandas_cols}")
        return False

    # 3. Ordenar ambos DataFrames si es posible
    if id_column and id_column in pandas_df.columns and id_column in sql_df.columns:
        pandas_df = pandas_df.sort_values(id_column).reset_index(drop=True)
        sql_df = sql_df.sort_values(id_column).reset_index(drop=True)
    else:
        # Si no hay columna id, ordenar por todas las columnas
        pandas_df = pandas_df.sort_values(list(pandas_df.columns)).reset_index(drop=True)
        sql_df = sql_df.sort_values(list(sql_df.columns)).reset_index(drop=True)

    # 4. Mostrar las primeras filas de cada uno para inspección visual
    print("DataFrame de Pandas:")
    display(pandas_df.head())
    print("\nDataFrame SQL:")
    display(sql_df.head())

    # 5. Comprobar si los valores son aproximadamente iguales (manejar punto flotante)
    for col in pandas_df.columns:
        pandas_col = pandas_df[col]
        sql_col = sql_df[col]

        # Convertir al mismo tipo para comparación
        if pandas_col.dtype != sql_col.dtype:
            # Intentar convertir ambos a cadena para comparación
            pandas_str = pandas_col.astype(str)
            sql_str = sql_col.astype(str)

            if not (pandas_str == sql_str).all():
                print(f"Los valores difieren en la columna: {col}")
                return False

    print("✅ Comparación de DataFrames exitosa!")
    return True

### Ejercicio 1: Filtrar datos por artista

In [ ]:
df.loc[df["artist_name"] == "Bon Iver"]

In [ ]:
# @title Solución con Pandas
my_artist_df_pandas = df.loc[df["artist_name"] == "Bon Iver"]

In [ ]:
my_artist_df_pandas

In [ ]:
# @title Ejercicio SQL
# TODO: Filtrar el conjunto de datos por artista usando SQL (BigQuery)
# Usar el comando mágico %%bigquery para consultar la tabla raw_data.spotify_full_dataset

%%bigquery my_artist_df --project $PROJECT_ID
SELECT
*
FROM
`raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'

In [ ]:
# @title Solución SQL
%%bigquery my_artist_df_sql --project $PROJECT_ID
SELECT *
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'

In [ ]:
# @title Comparar Pandas vs SQL
# Comparar los resultados del filtrado con Pandas y SQL
common_columns = list(set(my_artist_df_pandas.columns) & set(my_artist_df_sql.columns))
compare_results(my_artist_df_pandas[common_columns], my_artist_df_sql[common_columns])

### Ejercicio 2: ¿Cómo han evolucionado las características de las canciones a lo largo de los años?

In [ ]:
# @title Ejercicio con Pandas
# TODO: Calcular la evolución de las características a lo largo del tiempo usando Pandas
# Agrupar por año y calcular la media de las características numéricas
features_by_year_pandas =

In [ ]:
# @title Solución con Pandas
# Extraer características numéricas del dataframe del artista
numeric_features = my_artist_df_pandas.select_dtypes(np.number)
# Agrupar por año y calcular la media de cada característica
features_by_year_pandas = numeric_features.groupby("year", as_index=False).mean()

In [ ]:
# @title Ejercicio SQL
# TODO: Calcular la evolución de las características a lo largo del tiempo usando SQL (BigQuery)
# Escribir una consulta que agrupe por año y calcule el promedio de cada característica de sonido

%%bigquery features_by_year_sql --project $PROJECT_ID
SELECT
year,
AVG(acousticness) AS acousticness,
AVG(danceability) AS danceability,
AVG(energy) AS energy,
AVG(instrumentalness) AS instrumentalness,
AVG(liveness) AS liveness,
AVG(valence) AS valence
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY year

In [ ]:
# @title Solución SQL
%%bigquery features_by_year_sql --project $PROJECT_ID
SELECT
year,
AVG(acousticness) AS acousticness,
AVG(danceability) AS danceability,
AVG(energy) AS energy,
AVG(instrumentalness) AS instrumentalness,
AVG(liveness) AS liveness,
AVG(valence) AS valence,
AVG(loudness) AS loudness,
AVG(speechiness) AS speechiness,
AVG(tempo) AS tempo,
AVG(duration_ms) AS duration_ms,
AVG(key) AS key,
AVG(mode) AS mode,
AVG(popularity) AS popularity
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY year

In [ ]:
# @title Comparar Pandas vs SQL
# Comparar los resultados de Pandas y SQL para la evolución de características
# Enfocarse en las características clave de sonido para la comparación
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']
# Añadir 'year' a las columnas de comparación ya que es nuestra clave de agrupación
compare_columns = ['year'] + sound_features
compare_results(features_by_year_pandas[compare_columns], features_by_year_sql[compare_columns])

In [ ]:
# Visualizar la evolución de las características a lo largo del tiempo
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']
fig = px.line(features_by_year_sql, x='year', y=sound_features, height=1000, width=1800,
             title="Evolución de las características de sonido a lo largo del tiempo para Bon Iver")
fig.show()

### Ejercicio 3: ¿En qué años publicaron sus canciones (álbumes) y cuántas canciones se lanzaron?

In [ ]:
# @title Ejercicio con Pandas
# TODO: ¿En qué años publicaron sus canciones (álbumes) y cuántas canciones se lanzaron? (usar num_songs como el contador de num_songs)

songs_by_year_df_pandas =

In [ ]:
# @title Solución con Pandas
# TODO: ¿En qué años publicaron sus canciones (álbumes) y cuántas canciones se lanzaron? (usar num_songs como el contador de num_songs)

songs_by_year_df_pandas = my_artist_df_pandas.groupby("year").size().reset_index(name='num_songs').sort_index(ascending=True)

In [ ]:
# @title Ejercicio SQL
# Usar el comando mágico %%bigquery para consultar la tabla raw_data.spotify_full_dataset
# (usar num_songs como el contador de num_songs)

%%bigquery songs_by_year_df_sql --project $PROJECT_ID
SELECT
year,
COUNT(*) AS num_songs
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY year

In [ ]:
# @title Solución SQL
%%bigquery songs_by_year_df_sql --project $PROJECT_ID
SELECT
  year,
  COUNT(*) AS num_songs
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY year

In [ ]:
# @title Comparar Pandas vs SQL
# Comparar los resultados del filtrado con Pandas y SQL
common_columns = list(set(songs_by_year_df_pandas.columns) & set(songs_by_year_df_sql.columns))
compare_results(songs_by_year_df_pandas[common_columns], songs_by_year_df_sql[common_columns])

### Ejercicio 4: ¿Cuál es la canción más popular de tu artista?

In [ ]:
# @title Ejercicio con Pandas
# TODO: ¿Cuál es la canción más popular del artista?
most_popular_song_df_pandas = # TODO: usar la función nlargest()


In [ ]:
# @title Solución con Pandas
most_popular_song_df_pandas = my_artist_df_pandas.nlargest(1, 'popularity')

In [ ]:
# @title Ejercicio SQL
# TODO: Filtrar el conjunto de datos por artista usando SQL (BigQuery)
# Usar el comando mágico %%bigquery para consultar la tabla raw_data.spotify_full_dataset

%%bigquery most_popular_song_df_sql --project $PROJECT_ID
SELECT
*
FROM
`raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
ORDER BY popularity DESC
LIMIT 1

In [ ]:
# @title Solución SQL
%%bigquery most_popular_song_df_sql --project $PROJECT_ID
SELECT
  *
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
ORDER BY popularity DESC
LIMIT 1

In [ ]:
most_popular_song_df_sql

In [ ]:
# @title Comparar Pandas vs SQL
# Comparar los resultados del filtrado con Pandas y SQL
common_columns = list(set(most_popular_song_df_pandas.columns) & set(most_popular_song_df_sql.columns))
compare_results(most_popular_song_df_pandas[common_columns], most_popular_song_df_sql[common_columns])

### Ejercicio 5: ¿Cuál ha sido el año con la mayor energía?

In [ ]:
# @title Ejercicio con Pandas
# TODO: ¿En qué año se lanzaron las canciones con los niveles de energía promedio más altos?

# usar las funciones groupby y agg().
highest_energy_df_pandas =

In [ ]:
# @title Solución con Pandas
# TODO: ¿En qué año se lanzaron las canciones con los niveles de energía promedio más altos?
highest_energy_df_pandas = my_artist_df_pandas.groupby("year").agg({"energy": "mean"}).sort_values(by="energy", ascending=False).head(1).rename(columns={"energy": "avg_energy"})

In [ ]:
# @title Ejercicio SQL
# TODO: Filtrar el conjunto de datos por artista usando SQL (BigQuery)
# Usar el comando mágico %%bigquery para consultar la tabla raw_data.spotify_full_dataset

%%bigquery highest_energy_df_sql --project $PROJECT_ID
SELECT
year,
AVG(energy) AS avg_energy
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY avg_energy DESC
LIMIT 1

In [ ]:
# @title Solución SQL
%%bigquery highest_energy_df_sql --project $PROJECT_ID

SELECT
  year,
  AVG(energy) AS avg_energy
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY avg_energy DESC
LIMIT 1

In [ ]:
# @title Comparar Pandas vs SQL
# Comparar los resultados del filtrado con Pandas y SQL
common_columns = list(set(highest_energy_df_pandas.columns) & set(highest_energy_df_sql.columns))
compare_results(highest_energy_df_pandas[common_columns], highest_energy_df_sql[common_columns])

### Ejercicio 6: Correlación de características con la popularidad

In [ ]:
# @title Ejercicio con Pandas
# TODO: ¿Qué característica tiene la mayor correlación con la popularidad de la canción?

# Filtrar a columnas numéricas
df_bon_iver_numeric_columns = my_artist_df_pandas.select_dtypes(include=np.number).columns
df_bon_iver_numeric = my_artist_df_pandas[df_bon_iver_numeric_columns]

# usar la función corr(). ignorar los índices de popularidad y año, ordenar y limitar a 1
feature_corr_with_target_df_pandas =

In [ ]:
# @title Solución con Pandas
# TODO: ¿Qué característica tiene la mayor correlación con la popularidad de la canción?

# Filtrar a columnas numéricas
df_bon_iver_numeric_columns = my_artist_df_pandas.select_dtypes(include=np.number).columns
df_bon_iver_numeric = my_artist_df_pandas[df_bon_iver_numeric_columns]

# Primero calcular la matriz de correlación
correlation_matrix = df_bon_iver_numeric.corr()

# Extraer solo la columna de popularidad y eliminar la propia popularidad y cualquier otra columna que se desee excluir
feature_correlations = correlation_matrix.drop(['popularity', 'year'], axis=0)[['popularity']]

# Añadir una columna de correlación absoluta para ordenar
feature_correlations['abs_correlation'] = feature_correlations['popularity'].abs()

# Ordenar por correlación absoluta (descendente) y obtener la característica principal
top_feature = feature_correlations.sort_values(by='abs_correlation', ascending=False).head(1)

# Restablecer el índice para que el nombre de la característica sea una columna
feature_corr_with_target_df_pandas = top_feature.reset_index()

# Renombrar la columna de índice a 'feature'
feature_corr_with_target_df_pandas.rename(columns={'index': 'feature'}, inplace=True)

# Si es necesario, eliminar la columna abs_correlation para que coincida con la salida SQL
feature_corr_with_target_df_pandas = feature_corr_with_target_df_pandas[['feature', 'popularity']]

# Renombrar popularity para que coincida con la salida SQL
feature_corr_with_target_df_pandas.rename(columns={'popularity': 'correlation_with_popularity'}, inplace=True)


In [ ]:
feature_corr_with_target_df_pandas

In [ ]:
# @title Ejercicio SQL
# TODO: Filtrar el conjunto de datos por artista usando SQL (BigQuery)
# Usar el comando mágico %%bigquery para consultar la tabla raw_data.spotify_full_dataset

%%bigquery my_artist_df --project $PROJECT_ID
SELECT
*
FROM
`raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'


In [ ]:
# @title Solución SQL
%%bigquery feature_corr_with_target_df_sql --project $PROJECT_ID

WITH all_correlations AS (
  SELECT
    'danceability' AS feature,
    CORR(danceability, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'energy' AS feature,
    CORR(energy, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'acousticness' AS feature,
    CORR(acousticness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'instrumentalness' AS feature,
    CORR(instrumentalness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'liveness' AS feature,
    CORR(liveness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'valence' AS feature,
    CORR(valence, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'tempo' AS feature,
    CORR(tempo, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'loudness' AS feature,
    CORR(loudness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'speechiness' AS feature,
    CORR(speechiness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'duration_ms' AS feature,
    CORR(duration_ms, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'key' AS feature,
    CORR(key, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'mode' AS feature,
    CORR(mode, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'
)

SELECT
  feature,
  correlation_with_popularity
FROM all_correlations
ORDER BY ABS(correlation_with_popularity) DESC
LIMIT 1

In [ ]:
# @title Comparar Pandas vs SQL
# Comparar los resultados del filtrado con Pandas y SQL
common_columns = list(set(feature_corr_with_target_df_pandas.columns) & set(feature_corr_with_target_df_sql.columns))
compare_results(feature_corr_with_target_df_pandas[common_columns], feature_corr_with_target_df_sql[common_columns])